In [1]:
campaign_id = 1
manager_ethereum_address = '0xD3e9D60e4E4De615124D5239219F32946d10151D'

In [2]:
import mysql.connector as mysql
import datetime
import pandas as pd

In [3]:
# load api key
secret = {}
with open('secret_staging.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")


In [4]:
# load list of handles that must be rewarded
handles = list(pd.read_csv('handles.csv', header=None, names=['handle'])['handle'])
print(f"{len(handles)} handles loaded from CSV")

50 handles loaded from CSV


In [5]:
# load handles already rewarded for this campaign
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT twitter_handle FROM rewards where campaign_id = {campaign_id} and manager_ethereum_address = "{manager_ethereum_address}";'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()

handles_done = []
for r in records:
    handles_done.append(r[0])
    
print(f"{len(handles_done)} handles already rewarded for this campaign")

0 handles already rewarded for this campaign


In [31]:
for handle in handles:
    
    if handle not in handles_done:
        print(f"rewarding {handle}")
        
        # write rewards to database (with null blockchainwritetime)
        db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
        cursor = db.cursor()
        query = "INSERT INTO rewards (campaign_id, twitter_handle, manager_ethereum_address) VALUES (%s, %s, %s);"
        values = (campaign_id, handle, manager_ethereum_address)
        cursor.execute(query, values)
        db.commit()
        print(cursor.rowcount, "record inserted")
        cursor.close()
        db.close()
    else:
        print(f"{handle} already rewarded")

archipelagoai already rewarded
rewarding alxcnwy
1 record inserted
